# Proyecto: Análisis de mercado inmobiliario
<font size="+2">
    Desarrollado por David Mora <br>
    Segundo avance 
</font>

<font size="+1">Consigna</font><br>
En este proyecto profundizarás lo desarrollado en el proyecto 01 (“Primer modelo de Machine Learning”). El objetivo es aplicar las técnicas incorporadas (Transformación de Datos, Optimización de Hiperparámetros, Modelos Avanzados, etc.) para generar un modelo que tenga un mejor desempeño que el modelo generado en el proyecto anterior. Luego, interpreta ese modelo para responder la siguiente pregunta: ¿qué podemos aprender de nuestro problema estudiando el modelo que generamos?

Iniciare importando las bibliotecas que necesitaré y cargando el dataset de la primer entrega

In [86]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import time
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor

path='./DS_Proyecto_01_Datos_Properati.csv'
df_properati=pd.read_csv(path)

La siguiente celda hace que los dataframes de pandas se muestren solo con tres cifras decimales.

In [87]:
pd.options.display.float_format = "{:,.3f}".format

In [88]:
df_properati.head(1)
#len(df_properati)

,start_date,end_date,created_on,lat,lon,l1,l2,l3,rooms,bedrooms,bathrooms,surface_total,surface_covered,price,currency,title,description,property_type,operation_type
0,2019-10-17,2019-12-23,2019-10-17,-34.606,-58.385,Argentina,Capital Federal,San Cristobal,7.000,7.000,2.000,140.000,140.000,"153,000.000",USD,"***Venta semipiso centro, ideal hostel*****",DESCRIPCION DE LA PROPIEDAD: Departamento de 1...,Departamento,Venta


Ahora hare una revision de la cantidad de NaN por filas y por columnas. Lo hare mediante una funcion para poder comparar la cantidad de NaN que hay pendiendo de los cambios que se hagan al data frame.

In [192]:
def dfnans (df,imprimir='y'):
    
    #Conteto total de nan
    total=df.isnull().sum().sum()
    print ('El total de NaN en el dataframe es', total, 'corresponde al ',round(total*100/(19*len(df)),3),'% del total de los datos')
    
    #Seccion de conteo por columnas
    nan_col=df.isnull().sum().reset_index(name='Conteo por columnas')
    nan_col.rename(columns={'index': 'Atributo'}, inplace=True)
    nan_col.set_index('Atributo', inplace=True)
    nan_col['%']=(nan_col['Conteo por columnas']*100/len(df)).apply(lambda x:round(x,3))
    nan_col.append(nan_col.sum().rename('Total'))
    
    #seccion de conteo por filas
    nan_row=df.isnull().sum(axis=1).to_frame()
    nan_row['Conteo']=1
    nan_row.rename(columns={0: 'Numero de NaN en fila'}, inplace=True)
    count_nan_col=nan_row.groupby('Numero de NaN en fila').sum()
    count_nan_col['%']=(count_nan_col['Conteo']*100/len(df)).apply(lambda x:round(x,3))
    count_nan_col.append(count_nan_col.sum().rename('Total'), ignore_index=True)
    
    #Codigo para seleccionar si queremos que se impriman los resultados automaticamente, por defecto se imprimen, para no 
    #imprimir se debe colocar cualquier caracter
    if imprimir=='y' or imprimir=='Y':
        print(nan_col)
        print(count_nan_col)
    
    return total, nan_col, count_nan_col

In [193]:
dfnans(df_properati);

El total de NaN en el dataframe es 67982 corresponde al  2.44 % del total de los datos
                 Conteo por columnas      %
Atributo                                   
start_date                         0  0.000
end_date                           0  0.000
created_on                         0  0.000
lat                             9925  6.767
lon                             9959  6.791
l1                                 0  0.000
l2                                 0  0.000
l3                                 0  0.000
rooms                              0  0.000
bedrooms                           0  0.000
bathrooms                       5957  4.062
surface_total                  20527 13.996
surface_covered                21614 14.737
price                              0  0.000
currency                           0  0.000
title                              0  0.000
description                        0  0.000
property_type                      0  0.000
operation_type                   

In [91]:
print(len(df_properati[df_properati.l2=='Capital Federal']))
print(len(df_properati[df_properati.l2=='Bs.As. G.B.A. Zona Sur']))
print(len(df_properati[df_properati.l2=='Bs.As. G.B.A. Zona Norte']))
print(len(df_properati[df_properati.l2=='Bs.As. G.B.A. Zona Oeste']))

92539
17173
24282
12666


No se ultilizara el data frame con el filtrado propuesto para la parte tres de la primera entrega, considerare otros filtros para esta segunda entrega (los mas importantes los definire para la deteccion y eliminacion de outliers). Por ahora se limitara el data frame y en consecuencia los modelos del estudio solo a las inmuebles del tipo departemaneto, PH y casa. De todas las columnas del dataframe original descartaré las columnas start_date, end_date, created_on, l1, currency, title, descripcion, operation_type.

Justificar por que eliminar esas columnas...

A diferencia del filtrado para la parte final de la primera entrega en la que se trabajó solo con inmuebles de capital federal, en este caso trabajare con inmbuebles en todas las zonas de Buenos Aires, incluidas las zonas sur, norte y oeste. Esto hara que tengamos un dataframe mas grande pero del que podremos inferir precios de inmuebles de mas barrios.  

In [92]:
#Seleccion de propiedades en Capital Federal y cuyo tipo de propiedad es Departamento, PH o Casa.
#df=df_properati[(df_properati['l2']=='Bs.As. G.B.A. Zona Norte') & ((df_properati['property_type']=='Departamento') | (df_properati['property_type']=='Casa') | (df_properati['property_type']=='PH'))]
df=df_properati[(df_properati['property_type']=='Departamento') | (df_properati['property_type']=='Casa') | (df_properati['property_type']=='PH')]

#Se descartan las columnas start_date, end_date, created_on, l1, currency, title, descripcion, operation_type
df=df.drop(columns=['start_date', 'end_date', 'created_on', 'l1', 'currency', 'title', 'description', 'operation_type'])

#Se eliminaran las instancias que tengan surface_covered mayor a surface_total
df=df.drop(df[df.surface_covered > df.surface_total].index)

#Se eliminaran las instancias que tengan superficie total o cubierta menor a 15
df=df.drop(df[(df.surface_total<=15) | (df.surface_covered<=15)].index)

df.reset_index(drop=True, inplace=True)
df.head(1)

,lat,lon,l2,l3,rooms,bedrooms,bathrooms,surface_total,surface_covered,price,property_type
0,-34.606,-58.385,Capital Federal,San Cristobal,7.000,7.000,2.000,140.000,140.000,"153,000.000",Departamento


In [93]:
print('El tamaño del dataframe en este momento es de {} filas y {} columnas.'.format(df.shape[0],df.shape[1]))

El tamaño del dataframe en este momento es de 141547 filas y 11 columnas.


## PARTE A - Transformación de Datos

Elige cuáles de las siguientes tareas son apropiadas para su dataset. Implementa las transformaciones que elegiste. Es importante que justifiques por qué las haces:

- Detección y eliminación de Outliers
- Encoding
- Imputación de valores faltantes
- Escalado de datos
- Generación de nuevas variables predictoras/reducción de dimensionalidad (SVD/PCA).

Vuelve a entrenar el modelo implementado en la Entrega 01 - en particular, el árbol de decisión - con este nuevo dataset transformado. Evalúa su desempeño a partir del dataset obtenido luego de transformar los datos. ¿Hay una mejora en su desempeño? Compara con el desempeño obtenido en el proyecto 01. Sea cual sea la respuesta, intenta explicar a qué se debe.

NOTA: en la Parte A debes implementar al menos tres de las transformaciones de datos propuestas

Deteccion y eliminacion de Outliers

Encoding

Imputacion

Escalado

Generación de nuevas variables / reducción de dimensionalidad

### Deteccion y eliminación de outliers
Durante la primera entrega ya se habia trabajado con la deteccion y eliminacion de outliers, sin este proceso era imposible obtener los histogramas de forma entendible. En este caso, ya que posteriormente se va a aplicar encoding al atributo de property_type, hare deteccion y eliminacion de outliers segregando el tipo de propiedad departamento, casa y PH.

En la siguiete celda se uso la funcion describe para definir una funcion que calcula el rango intercuertilico (RIC), limite inferior (lim_inf) y limite superior (lim_sup) de cada atributo y lo devuelve en un dataframe.

In [94]:
#los argumentos son df: dataframe al que se le aplica la fucncion.
def RICdescribe(df):
    df_describe=df.describe().T
    df_describe['RIC']=df_describe['75%']-df_describe['25%']
    df_describe['lim_inf']=df_describe['25%']-1.5*df_describe['RIC']
    df_describe['lim_sup']=df_describe['75%']+1.5*df_describe['RIC']
    return df_describe.T

A continuacion creare una funcion que cambia los valores atipicos por NaN

In [95]:
def NaN_outliers(df, RICdesc):
    dff=df.copy() #se crea una copia del dataframe argumento
    for i in RICdesc.columns:
        dff[i].where((dff[i] > RICdesc.loc['lim_inf'][i]) & (dff[i] < RICdesc.loc['lim_sup'][i]), inplace=True)
    return dff

#### Outliers en general

In [96]:
RICdesc=RICdescribe(df)
RICdesc

,lat,lon,rooms,bedrooms,bathrooms,surface_total,surface_covered,price
count,"131,986.000","131,955.000","141,547.000","141,547.000","137,355.000","122,994.000","122,638.000","141,547.000"
mean,-34.603,-58.465,3.089,1.993,1.596,182.866,103.707,"237,899.215"
std,0.476,0.494,1.378,1.121,0.883,"1,445.091",736.907,"285,948.655"
min,-35.151,-180.000,1.000,0.000,1.000,16.000,16.000,"6,000.000"
25%,-34.626,-58.503,2.000,1.000,1.000,52.000,47.000,"112,000.000"
50%,-34.601,-58.443,3.000,2.000,1.000,78.000,68.000,"166,000.000"
75%,-34.574,-58.408,4.000,3.000,2.000,137.000,107.000,"264,187.500"
max,85.051,-57.806,35.000,15.000,14.000,"169,000.000","126,062.000","32,434,232.000"
RIC,0.052,0.095,2.000,2.000,1.000,85.000,60.000,"152,187.500"
lim_inf,-34.704,-58.645,-1.000,-2.000,-0.500,-75.500,-43.000,"-116,281.250"


Se puede observar que a excepcion de los atributos lat y lon los valores minimos del resto de atributos estan dentro del intervalo de aceptacion. Mientras que para todos los atributos sus respectivos valores maximos estan por fuera del intervalo de valores aceptados. A los valores que estan por fuera del intervalo de aceptacion se le asignara NaN.

In [97]:
df_nanol=NaN_outliers(df, RICdesc)

In [98]:
dfnans(df);

El total de NaN en el dataframe es 60807 corresponde al  2.261 % del total de los datos
                 Conteo por columnas      %
Atributo                                   
lat                             9561  6.755
lon                             9592  6.777
l2                                 0  0.000
l3                                 0  0.000
rooms                              0  0.000
bedrooms                           0  0.000
bathrooms                       4192  2.962
surface_total                  18553 13.107
surface_covered                18909 13.359
price                              0  0.000
property_type                      0  0.000
                       Conteo      %
Numero de NaN en fila               
0                      112282 79.325
1                        2620  1.851
2                       23069 16.298
3                        2466  1.742
4                         899  0.635
5                         211  0.149


In [99]:
dfnans(df_nanol);

El total de NaN en el dataframe es 146699 corresponde al  5.455 % del total de los datos
                 Conteo por columnas      %
Atributo                                   
lat                            35968 25.411
lon                            26637 18.818
l2                                 0  0.000
l3                                 0  0.000
rooms                           2981  2.106
bedrooms                        1040  0.735
bathrooms                       9580  6.768
surface_total                  31336 22.138
surface_covered                28636 20.231
price                          10521  7.433
property_type                      0  0.000
                       Conteo      %
Numero de NaN en fila               
0                       76981 54.385
1                       16156 11.414
2                       27938 19.738
3                       11936  8.433
4                        5391  3.809
5                        1933  1.366
6                         912  0.644
7     

#### Outliers por departamento

In [100]:
df_depa=df[df.property_type=='Departamento']

In [101]:
RICdescribe_depa=RICdescribe(df_depa)
RICdescribe_depa

,lat,lon,rooms,bedrooms,bathrooms,surface_total,surface_covered,price
count,"98,648.000","98,636.000","106,454.000","106,454.000","103,766.000","92,927.000","92,565.000","106,454.000"
mean,-34.594,-58.454,2.758,1.759,1.511,99.399,81.915,"237,465.670"
std,0.389,0.404,1.146,0.950,0.814,646.834,436.638,"304,436.236"
min,-35.121,-180.000,1.000,0.000,1.000,16.000,16.000,"6,000.000"
25%,-34.616,-58.481,2.000,1.000,1.000,48.000,44.000,"110,000.000"
50%,-34.598,-58.438,3.000,2.000,1.000,67.000,60.000,"160,000.000"
75%,-34.574,-58.407,3.000,2.000,2.000,99.000,85.000,"254,000.000"
max,85.051,-57.901,32.000,15.000,14.000,"108,960.000","104,890.000","32,434,232.000"
RIC,0.042,0.074,1.000,1.000,1.000,51.000,41.000,"144,000.000"
lim_inf,-34.680,-58.593,0.500,-0.500,-0.500,-28.500,-17.500,"-106,000.000"


In [102]:
dfnans(df_depa);

El total de NaN en el dataframe es 45728 corresponde al  2.261 % del total de los datos
                 Conteo por columnas      %
Atributo                                   
lat                             7806  7.333
lon                             7818  7.344
l2                                 0  0.000
l3                                 0  0.000
rooms                              0  0.000
bedrooms                           0  0.000
bathrooms                       2688  2.525
surface_total                  13527 12.707
surface_covered                13889 13.047
price                              0  0.000
property_type                      0  0.000
                       Conteo      %
Numero de NaN en fila               
0                       84273 79.164
1                        1726  1.621
2                       18160 17.059
3                        1609  1.511
4                         575  0.540
5                         111  0.104


In [103]:
df_depa_nanol=NaN_outliers(df_depa, RICdescribe_depa);

In [104]:
dfnans(df_depa_nanol);

El total de NaN en el dataframe es 108074 corresponde al  5.343 % del total de los datos
                 Conteo por columnas      %
Atributo                                   
lat                            24337 22.862
lon                            18632 17.502
l2                                 0  0.000
l3                                 0  0.000
rooms                           6023  5.658
bedrooms                        3940  3.701
bathrooms                       5760  5.411
surface_total                  20068 18.851
surface_covered                20514 19.270
price                           8800  8.266
property_type                      0  0.000
                       Conteo      %
Numero de NaN en fila               
0                       59012 55.434
1                       11755 11.042
2                       22447 21.086
3                        6527  6.131
4                        3462  3.252
5                        1915  1.799
6                        1016  0.954
7     

#### Outliers por propiedad horizontal PH

In [105]:
df_PH=df[df.property_type=='PH']

In [106]:
RICdescribe_PH=RICdescribe(df_PH)
RICdescribe_PH

,lat,lon,rooms,bedrooms,bathrooms,surface_total,surface_covered,price
count,"13,761.000","13,756.000","14,181.000","14,181.000","14,004.000","14,066.000","14,026.000","14,181.000"
mean,-34.617,-58.455,3.528,2.416,1.483,173.359,117.198,"188,577.963"
std,0.074,0.110,1.338,1.155,0.745,"2,033.645","1,519.019","112,895.715"
min,-35.121,-58.938,1.000,0.000,1.000,20.000,17.000,"13,000.000"
25%,-34.636,-58.504,3.000,2.000,1.000,70.000,56.000,"109,000.000"
50%,-34.609,-58.458,3.000,2.000,1.000,106.000,80.000,"160,000.000"
75%,-34.588,-58.421,4.000,3.000,2.000,151.000,118.000,"248,000.000"
max,-34.226,-57.887,35.000,15.000,11.000,"126,062.000","126,062.000","1,750,000.000"
RIC,0.049,0.084,1.000,1.000,1.000,81.000,62.000,"139,000.000"
lim_inf,-34.710,-58.630,1.500,0.500,-0.500,-51.500,-37.000,"-99,500.000"


In [107]:
df_PH_nanol= (df_PH, RICdescribe_PH)

In [108]:
dfnans(df_PH);

El total de NaN en el dataframe es 1292 corresponde al  0.48 % del total de los datos
                 Conteo por columnas     %
Atributo                                  
lat                              420 2.962
lon                              425 2.997
l2                                 0 0.000
l3                                 0 0.000
rooms                              0 0.000
bedrooms                           0 0.000
bathrooms                        177 1.248
surface_total                    115 0.811
surface_covered                  155 1.093
price                              0 0.000
property_type                      0 0.000
                       Conteo      %
Numero de NaN en fila               
0                       13431 94.711
1                         269  1.897
2                         441  3.110
3                          23  0.162
4                          13  0.092
5                           4  0.028


In [109]:
dfnans(df_PH_nanol);

El total de NaN en el dataframe es 7263 corresponde al  2.696 % del total de los datos
                 Conteo por columnas      %
Atributo                                   
lat                             1903 13.419
lon                             1299  9.160
l2                                 0  0.000
l3                                 0  0.000
rooms                            962  6.784
bedrooms                         536  3.780
bathrooms                        476  3.357
surface_total                   1037  7.313
surface_covered                  642  4.527
price                            408  2.877
property_type                      0  0.000
                       Conteo      %
Numero de NaN en fila               
0                       10171 71.723
1                        1886 13.299
2                        1518 10.704
3                         289  2.038
4                         170  1.199
5                          92  0.649
6                          51  0.360
7       

#### Outliers por casa

In [110]:
df_casa=df[df.property_type=='Casa']

In [111]:
RICdescribe_casa=RICdescribe(df_casa)
RICdescribe_casa

,lat,lon,rooms,bedrooms,bathrooms,surface_total,surface_covered,price
count,"19,577.000","19,563.000","20,912.000","20,912.000","19,585.000","16,001.000","16,047.000","20,912.000"
mean,-34.640,-58.528,4.474,2.902,2.128,675.958,217.617,"273,552.301"
std,0.870,0.899,1.536,1.331,1.111,"3,115.055","1,008.870","264,749.088"
min,-35.151,-180.000,1.000,0.000,1.000,25.000,21.000,"10,000.000"
25%,-34.755,-58.683,3.000,2.000,1.000,201.000,110.000,"130,000.000"
50%,-34.641,-58.524,4.000,3.000,2.000,307.000,160.000,"208,000.000"
75%,-34.540,-58.402,5.000,4.000,3.000,589.000,230.000,"330,000.000"
max,85.051,-57.806,26.000,15.000,13.000,"169,000.000","75,792.000","11,111,111.000"
RIC,0.215,0.281,2.000,2.000,2.000,388.000,120.000,"200,000.000"
lim_inf,-35.077,-59.104,0.000,-1.000,-2.000,-381.000,-70.000,"-170,000.000"


In [112]:
df_casa_nanol=NaN_outliers(df_casa, RICdescribe_casa)

In [113]:
dfnans(df_casa);

El total de NaN en el dataframe es 13787 corresponde al  3.47 % del total de los datos
                 Conteo por columnas      %
Atributo                                   
lat                             1335  6.384
lon                             1349  6.451
l2                                 0  0.000
l3                                 0  0.000
rooms                              0  0.000
bedrooms                           0  0.000
bathrooms                       1327  6.346
surface_total                   4911 23.484
surface_covered                 4865 23.264
price                              0  0.000
property_type                      0  0.000
                       Conteo      %
Numero de NaN en fila               
0                       14578 69.711
1                         625  2.989
2                        4468 21.366
3                         834  3.988
4                         311  1.487
5                          96  0.459


In [114]:
dfnans(df_casa_nanol);

El total de NaN en el dataframe es 18739 corresponde al  4.716 % del total de los datos
                 Conteo por columnas      %
Atributo                                   
lat                             1365  6.527
lon                             1886  9.019
l2                                 0  0.000
l3                                 0  0.000
rooms                            757  3.620
bedrooms                         139  0.665
bathrooms                       1567  7.493
surface_total                   6101 29.175
surface_covered                 5665 27.090
price                           1259  6.020
property_type                      0  0.000
                       Conteo      %
Numero de NaN en fila               
0                       11944 57.116
1                        2365 11.309
2                        4490 21.471
3                        1323  6.327
4                         571  2.730
5                         176  0.842
6                          41  0.196
7      

Al hacer las sumatorias de los Valores NaN antes de convertir los valores atipicos y despues de convertirlos se obtuvo que el total de la suma de los NaN por categorias Departamento, PH y Casa son 134076, mientras que sin discriminar por categorias es de 146699, esto nos muestra que es ventajoso discriminar segun el tipo de propiedad.

#### Unificacion y eliminacion por filas

Ahora se deben unificar los dataframes, que estaban separados por tipo de inmueble.

In [115]:
#El data frame resultante de la union sera df_nol (dataframe no outliers)
df_nol=df_depa_nanol.copy()
df_nol=df_nol.append(df_PH_nanol)
df_nol=df_nol.append(df_casa_nanol)
df_nol.sort_index(inplace=True)
#df_nol
print(df_nol.shape)

(141547, 11)


Ahora evaluamos la cantidad de NaN en este dataframe que contiene todos los tipos de propiedad:

In [116]:
dfnans(df_nol);

El total de NaN en el dataframe es 134076 corresponde al  4.985 % del total de los datos
                 Conteo por columnas      %
Atributo                                   
lat                            27605 19.502
lon                            21817 15.413
l2                                 0  0.000
l3                                 0  0.000
rooms                           7742  5.470
bedrooms                        4615  3.260
bathrooms                       7803  5.513
surface_total                  27206 19.220
surface_covered                26821 18.948
price                          10467  7.395
property_type                      0  0.000
                       Conteo      %
Numero de NaN en fila               
0                       81127 57.315
1                       16006 11.308
2                       28455 20.103
3                        8139  5.750
4                        4203  2.969
5                        2183  1.542
6                        1108  0.783
7     

Podemos ver que hay 86 filas cuyos valores numericos son todos NaN, considero que las filas con mas de 3 NaN deben ser eliminadas del dataframe esto porque tenian informacion faltante desde un principio y/o los valores que si tenian eran atipicos. Las filas que tienen mas de 3 NaN repreentan solo el 5.5% del total de las filas.

In [117]:
df_nol.dropna(thresh=df_nol.shape[1]-3,inplace=True)

Revisemos cuantos nans quedaron en el data frame:

In [118]:
dfnans(df_nol);

El total de NaN en el dataframe es 97333 corresponde al  3.831 % del total de los datos
                 Conteo por columnas      %
Atributo                                   
lat                            23962 17.919
lon                            18454 13.800
l2                                 0  0.000
l3                                 0  0.000
rooms                           3420  2.557
bedrooms                        1528  1.143
bathrooms                       4618  3.453
surface_total                  19969 14.933
surface_covered                19611 14.665
price                           5771  4.316
property_type                      0  0.000
                       Conteo      %
Numero de NaN en fila               
0                       81127 60.666
1                       16006 11.969
2                       28455 21.278
3                        8139  6.086


In [119]:
print('El dataframe tiene en este momento ',len(df_nol),'filas.')

El dataframe tiene en este momento  133727 filas.


In [120]:
df_nol.reset_index(inplace=True,drop=True)
#df_nol

### Encoding
Revisando la documentacion sobre encoders y de imputacion de scikitlearn me he dado cuenta que los atributos categoricos no pueden aportar su informacion para hacer imputacion debido a que los imputadores solo pueden imputar en base a los atributos numericos, por esta razon se decidio aplicar una codificacion (encoding) a los atributos categoricos antes de hacer la imputacion de valores faltantes.

En los atributos categoricos no hay ningun dato faltante por lo que a pesar de que se codifiquen no se tendra problema de obtener valores float al hacer imputacion.

In [121]:
from sklearn.preprocessing import OrdinalEncoder

#### Para zonas (l2)

Acontinuacion se hace la imputacion para la categoria l2 (zonas de Buenos Aires)

In [122]:
encl2 = OrdinalEncoder()
df_nol['l2']=encl2.fit_transform(df_nol['l2'].to_numpy().reshape(-1,1))

In [123]:
encl2.categories_

[array(['Bs.As. G.B.A. Zona Norte', 'Bs.As. G.B.A. Zona Oeste',
        'Bs.As. G.B.A. Zona Sur', 'Capital Federal'], dtype=object)]

#### Para barrios (l3)

Acontinuacion se hace la imputacion para la categoria l3 (barrios de Buenos Aires)

In [124]:
encl3 = OrdinalEncoder()
df_nol['l3']=encl3.fit_transform(df_nol['l3'].to_numpy().reshape(-1,1))

In [125]:
encl3.categories_

[array(['Abasto', 'Agronomía', 'Almagro', 'Almirante Brown', 'Avellaneda',
        'Balvanera', 'Barracas', 'Barrio Norte', 'Belgrano', 'Berazategui',
        'Boca', 'Boedo', 'Caballito', 'Catalinas', 'Cañuelas',
        'Centro / Microcentro', 'Chacarita', 'Coghlan', 'Colegiales',
        'Congreso', 'Constitución', 'Escobar', 'Esteban Echeverría',
        'Ezeiza', 'Florencio Varela', 'Flores', 'Floresta',
        'General Rodríguez', 'General San Martín', 'Hurlingham',
        'Ituzaingó', 'José C Paz', 'La Matanza', 'La Plata', 'Lanús',
        'Las Cañitas', 'Liniers', 'Lomas de Zamora', 'Malvinas Argentinas',
        'Marcos Paz', 'Mataderos', 'Merlo', 'Monserrat', 'Monte Castro',
        'Moreno', 'Morón', 'Nuñez', 'Once', 'Palermo', 'Parque Avellaneda',
        'Parque Centenario', 'Parque Chacabuco', 'Parque Chas',
        'Parque Patricios', 'Paternal', 'Pilar', 'Pompeya',
        'Presidente Perón', 'Puerto Madero', 'Quilmes', 'Recoleta',
        'Retiro', 'Saavedra', 'San 

#### Para tipo de propiedad

Acontinuacion se hace la imputacion para la categoria property_type (tipo de inmueble)

In [126]:
enc_PT = OrdinalEncoder()
df_nol['property_type']=enc_PT.fit_transform(df_nol['property_type'].to_numpy().reshape(-1,1))

In [127]:
enc_PT.categories_

[array(['Casa', 'Departamento', 'PH'], dtype=object)]

In [128]:
df_nol

,lat,lon,l2,l3,rooms,bedrooms,bathrooms,surface_total,surface_covered,price,property_type
0,-34.606,-58.385,3.000,63.000,NaN,NaN,2.000,140.000,140.000,"153,000.000",1.000
1,-34.624,-58.412,3.000,11.000,2.000,1.000,2.000,70.000,58.000,"159,000.000",2.000
2,-34.594,-58.427,3.000,48.000,2.000,1.000,1.000,45.000,45.000,"125,000.000",2.000
3,-34.581,-58.437,3.000,48.000,2.000,1.000,1.000,85.000,50.000,"295,000.000",2.000
4,NaN,NaN,2.000,33.000,2.000,1.000,1.000,50.000,35.000,"40,000.000",2.000
...,...,...,...,...,...,...,...,...,...,...,...
133722,NaN,NaN,2.000,9.000,2.000,1.000,1.000,54.000,48.000,"165,000.000",1.000
133723,NaN,NaN,0.000,55.000,4.000,3.000,2.000,164.000,90.000,"130,000.000",1.000
133724,NaN,NaN,0.000,55.000,1.000,1.000,1.000,50.000,35.000,"73,000.000",1.000
133725,NaN,NaN,0.000,55.000,2.000,1.000,1.000,38.000,38.000,"68,000.000",1.000


### Valores faltantes e imputacion
Luego de la asignacion de NaN a los outliers la cantidad de valores faltantes aumentara significativamente, para ver cuantos son los valores faltantes en este punto procederemos a usar nuevamente la funcion dfnans.

In [325]:
dfnans(df_nol,'n')[1]

El total de NaN en el dataframe es 97333 corresponde al  3.831 % del total de los datos


,Conteo por columnas,%
Atributo,,
lat,23962,17.919
lon,18454,13.800
l2,0,0.000
l3,0,0.000
rooms,3420,2.557
bedrooms,1528,1.143
bathrooms,4618,3.453
surface_total,19969,14.933
surface_covered,19611,14.665


Podemos ver que los atributos de latitud porcentaje de valores faltantes es aproximadamente 18% y para longitud 14%, para rooms, bedrooms y bathrooms es en promedio 2.4%, mientras que para surface_total y surface_covered esta es aproximadamente 15%, el atributo price, que es el que buscamos predecir tiene un 4.3% de valores faltantes.

Es importante recordar que muchos de estos valores faltantes aparecieron al aplicar la funcion NaN_outliers que les asignana NaN a los valores que estuvieran por fuera de los intervalos de validez y dado que en este caso se busca solo determinar que tipo de valores faltantes se tienen (MCAR, MAR o MCAR) se trabajara con el dataframe original de properati

Veamos como es la distribucion de valores fatantes en el dataframe original

In [326]:
dfnans(df_properati,'n')[1]

El total de NaN en el dataframe es 67982 corresponde al  2.44 % del total de los datos


,Conteo por columnas,%
Atributo,,
start_date,0,0.000
end_date,0,0.000
created_on,0,0.000
lat,9925,6.767
lon,9959,6.791
l1,0,0.000
l2,0,0.000
l3,0,0.000
rooms,0,0.000


Solo los atributos lat, lon, bathrooms, surface_covered y surface_total tienen valores faltantes.

#### MAR

A continuacion buscaremos si hay relacion entre los atributos categoricos l2, l3 y property_type y los atributos que tienen valores faltantes lat, lon, bathrooms, surface_total y surface_covered.

La funcion seraMAR se creo para determinar si los datos faltantes son del tipo MAR (Missing At Random) en este caso la falta de datos en alguna de la varibles en particular depende de las demas variables medidas

In [342]:
def seraMAR (df,atr,atr_nan,code):
    j={}
    
    j[atr]=df.surface_total.isnull().groupby([df[atr]]).sum().astype(int).reset_index(name='NaN_surface_total')[atr]
    #print(j)
    for i in range(len(atr_nan)):
        j[atr_nan[i]]=df[atr_nan[i]].isnull().groupby([df[atr]]).sum().astype(int).reset_index(name=atr_nan[i])[atr_nan[i]]
        j[atr_nan[i]+'%']=j[atr_nan[i]]*100/j[atr_nan[i]].sum()
   
    count=df[atr].value_counts()
    MAR=pd.DataFrame(j)
    MAR=MAR.set_index(atr).join(count)
    MAR[atr+'%']=MAR[atr]*100/MAR[atr].sum()
    
    if code=='porc':
    #Se presentan solo las columnas de porcentaje de nans:
        lista=list(range(1,2*len(atr_nan)+2,2))
        lis=[*lista[-1:],*lista[:-1]]
    if code=='todo':
    #Se presentan las columnas de conteo de nans y sus porcentajes:
        lista=list(range(2*len(atr_nan)+2))
        lis=[*lista[-2:],*lista[:-2]]
        
    MAR=MAR.iloc[:,lis].sort_values(by=[atr+'%'],ascending=False)

    return MAR

In [343]:
atr_nan=['lat','lon','bathrooms','surface_total','surface_covered']

##### MAR l2

En la siguiente tabla se puede comparar los porcentajes de valores faltantes por categoria l2 con respecto a los porcentajes del total de datos que hay en cada categoria de l2 (primer columna)

In [344]:
seraMAR(df_properati,'l2',atr_nan,'porc')

,l2%,lat%,lon%,bathrooms%,surface_total%,surface_covered%
l2,,,,,,
Capital Federal,63.098,46.287,46.270,42.337,45.642,45.646
Bs.As. G.B.A. Zona Norte,16.557,35.788,35.666,12.238,9.568,10.049
Bs.As. G.B.A. Zona Sur,11.709,9.783,9.750,29.360,27.442,27.163
Bs.As. G.B.A. Zona Oeste,8.636,8.141,8.314,16.065,17.348,17.142


La primera columna nos muestra la distribucion de datos con valores asignados en cada categoria de l2, si los valores faltantes fueran MCAR (Missing Completely At Random) estos porcentajes deberian ser similares en el resto de columnas, dicho esto podemos interpretar las siguientes situaciones, en la zona norte para los atributos lat y lon hay mas valores faltantes de lo que deberia haber (aproximadamente 20% mas), en las zonas sur y oeste para los atributos bathrooms, surface_covered y surface_total hay mas falores faltantes de lo que deberia haber (aproximadamente 16% y 9% mas), estos valores de mas corresponden a valores MAR.

##### MAR l3

En la siguiente tabla se puede comparar los porcentajes de valores faltantes por categoria l3 con respecto a los porcentajes del total de datos que hay en cada categoria de l3 (primer columna)

In [354]:
seraMAR(df_properati,'l3',atr_nan,'porc').sort_values(by=['lat%'],ascending=False)

,l3%,lat%,lon%,bathrooms%,surface_total%,surface_covered%
l3,,,,,,
Tigre,5.944,23.849,23.767,2.585,1.379,1.383
Palermo,8.914,11.023,11.005,3.307,3.025,3.040
Belgrano,4.729,6.045,6.055,1.528,4.058,4.053
Recoleta,4.035,5.602,5.583,1.679,1.559,1.527
Pilar,2.529,4.665,4.649,1.729,1.656,1.758
...,...,...,...,...,...,...
Parque Avellaneda,0.184,0.010,0.010,0.336,0.268,0.254
Villa Soldati,0.047,0.000,0.000,0.067,0.010,0.014
Villa Riachuelo,0.029,0.000,0.000,0.034,0.015,0.032


En este atributo es dificil ver los cambios en porcentajes debido que hay una cantidad de barrios muy grande (89) pero podemos ver que el barrio Tigre en el atributo lat y lon tiene un porcentaje mucho mayor (18% aproximadamente) que valores faltantes a lo correspondiente a faltantes tipo MCAR, el barrio Lomas de Zamora en el atributo bathrooms, surface_covered y surface_total tiene un porcentaje mayor (7% y 10% aproximadamente) que valores faltantes a lo correspondiente a faltantes tipo MCAR, estos faltantes adicionales son tipo MAR

##### MAR property_type

En la siguiente tabla se puede comparar los porcentajes de valores faltantes por categoria property_type con respecto a los porcentajes del total de datos que hay en cada categoria de property_type (primer columna)

In [356]:
seraMAR(df_properati,'property_type',atr_nan,'porc')

,property_type%,lat%,lon%,bathrooms%,surface_total%,surface_covered%
property_type,,,,,,
Departamento,73.180,78.841,78.693,45.258,66.142,64.259
Casa,14.674,13.552,13.646,22.444,23.954,22.509
PH,9.749,4.262,4.298,2.988,0.628,0.717
Lote,0.895,1.259,1.255,14.017,2.957,5.917
Oficina,0.449,0.312,0.311,1.880,1.286,1.300
Otro,0.255,0.332,0.351,2.149,0.750,1.217
Local comercial,0.222,0.272,0.281,2.871,1.466,1.402
Casa de campo,0.220,0.171,0.171,0.353,0.312,0.296
Depósito,0.181,0.796,0.793,3.844,1.252,1.194


En la anterior tabla podemos ver que los atributos lat y lon tienen porcentajes muy parecidos a los porcentajes de los valores de los que se tienen datos, difieren a lo sumo en un 6% en el caso en el que shay mas datos (Departamento). PAra el resto de atributos podemos ver que las propiedades que no son depattamento, casa o PH tienen porcentajes mucho mas altos de como deberian ser si fueran faltantes tipo MCAR. En particular esta el caso de las propiedades tipo lote y el atributo bathrooms (13% de mas), supongo que estos valores faltantes se deben a que la mayoria de lotes no incluyen baño y por alguna razón los usuarios de properati al momento del registro del inmueble, en vez de registrar cero baños, no se suministran la infromación, parecido puede pasar para los lotes y el atributo surface, se sabe que esta a la venta el lote pero no su superficie.

#### MNAR

Los valores faltante MNAR (Missing Not At Random) son aquellos datos en un atributo tal que son faltantes debido a valor de ese mismo atributo. Para este punto de nuestro dataframe podriamos decir que todas los atributos en los que hemos eliminado valores por ser outliers serian valores MNAR, porque por ejemplo en el atributo precio de las casas, todos los precios mayores a 630 mil dolares fueron eliminados. 

Ahora, antes de hacer la eliminacion de outliers habian datos MNAR?

Para resolver esta duda me remitire a los histogramas del primer proyecto, estos histogramas se hicieron sobre el dataframe original para el atributo bathrooms, habiendo eliminado los outliers y sin haber eliminado outliers, y para los atributos surface_covered y surface_total solamente luego de haber eliminado los outliers ya que la grafica no nos daba informacion si se incluian los outliers, es de notar que estoy mencionando solo los atributos que tienen valores faltantes en el dataframe original. En cualquiera de estos histogramas podemos ver que, de todo el dominio de valores que estan representados por el ancho de las barras, no hay ninguna barra que sea significativamente baja con respecto a las barras contiguas, y en el caso particular de los atributos surface_covered y surface_total que son atributos continuos si se usaran barras mas gruesas se veria un comportamiento mas estable. Estos observaciones sobre los histogramas me llevan a pensar que no hay valores MNAR en el dataframe original.

#### Imputación

In [146]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


X_test = df_nol[['lat','lon','rooms','bedrooms','bathrooms','surface_total','surface_covered']]
# the model learns that the second feature is double the first
#print(X_test)

In [147]:
imp = IterativeImputer(max_iter=10, random_state=0)
imp.fit(X_test)

C:\Users\David Mora\miniconda3\envs\datascience\lib\site-packages\sklearn\impute\_iterative.py:685: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn("[IterativeImputer] Early stopping criterion not"


IterativeImputer(random_state=0)

In [148]:
print(imp.transform(X_test))

[[-34.6058798  -58.3849491    3.95773546 ...   2.         140.
  140.        ]
 [-34.6240556  -58.4121102    2.         ...   2.          70.
   58.        ]
 [-34.593569   -58.427474     2.         ...   1.          45.
   45.        ]
 ...
 [-34.57950025 -58.47071953   1.         ...   1.          50.
   35.        ]
 [-34.60745097 -58.44257714   2.         ...   1.          38.
   38.        ]
 [-34.61057965 -58.44654216   3.         ...   1.          75.
   75.        ]]


In [149]:
from sklearn.impute import KNNImputer

In [150]:
imputer = KNNImputer(n_neighbors=2, weights="uniform")

In [151]:
inicio=time.time()
knn_imputer=imputer.fit_transform(df_nol)
fin=time.time()
print(fin-inicio)

842.2966604232788


In [ ]:
knn_imputer

## PARTE B - Modelos Avanzados

1. Elige dos de los modelos avanzados vistos Compara con el desempeño obtenido en el proyecto 01 (en el caso de regresión, considera una regresión lineal con atributos polinómicos y regularización). Entrénalos y evalúalos con sus argumentos por defecto. No te olvides de hacer un train/test split y usar Validación Cruzada.

2. Optimiza sus hiperparámetros mediante Validación Cruzada y Grid Search o Random Search.

3. Compara el desempeño de los nuevos modelos entre sí y con el modelo de la Parte A. ¿Cuál elegirías? Justifica.

NOTA: en la Parte B, al menos un modelo debe ser optimizado por Grid Search o Random Search; el otro puede ser optimizado por búsqueda manual (es decir, puedes dejar los mejores parámetros que encontraste probando).


## PARTE C - Interpretación de modelos

De acuerdo a lo que el modelo permite, responde algunas o todas las siguientes preguntas:

- ¿Qué variables fueron relevantes para el modelo para hacer una predicción? ¿Cuáles no? Si usaste una regresión lineal con regularización, presta atención a los parámetros (pendientes) obtenidas. Si usaste un modelo de ensamble en árboles, además de ver la importancia de cada atributo, también elige algunos árboles al azar y observa qué atributos considera importantes. ¿En qué se diferencian esos árboles? ¿Por qué? Finalmente, responde, ¿coincide con lo que esperabas a partir de tu experiencia con este dataset?

- ¿Cómo es la distribución de errores (regresión) o qué clases se confunden entre sí (clasificación)? ¿Dónde falla? ¿A qué se debe?

NOTA: en la Parte C, debes responder al menos una pregunta. Obviamente, ¡cuanto más hagas, más aprenderás y mejor será tu proyecto!

## DESAFÍO OPCIONAL

Aplica una técnica de Clustering sobre el dataset. Puedes combinar con técnicas de reducción de dimensionalidad para facilitar la visualización. ¿Qué clusters encuentras? ¿A qué pueden corresponder? Te dejamos preguntas que pueden servir como disparadoras: ¿qué barrios se parecen más entre sí?¿qué tipos de propiedades se parecen más entre sí?